In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

file_path = 'News (1).xlsx'
df = pd.read_excel(file_path)

# Hiển thị một vài dòng đầu tiên
print(df.head())

   Unnamed: 0    Bank                                               Link  \
0           0  ABBank  https://cafef.vn/abbank-giam-toi-15-nam-lai-su...   
1           1  ABBank  https://cafef.vn/abbank-va-maybank-tang-cuong-...   
2           2  ABBank  https://cafef.vn/16-ca-nhan-3-to-chuc-so-huu-g...   
3           3  ABBank  https://cafef.vn/abbank-dat-558-ty-dong-loi-nh...   
4           4  ABBank  https://cafef.vn/nhieu-uu-dai-danh-cho-khach-h...   

                                               Title  \
0  ABBank giảm tới 1,5%/năm lãi suất cho vay khác...   
1  Maybank tăng cường quan hệ hợp tác chiến lược,...   
2  16 cá nhân, 3 tổ chức sở hữu gần 67% vốn điều ...   
3  ABBANK đạt 558 tỷ đồng lợi nhuận trước thuế tr...   
4  Nhiều ưu đãi dành cho khách hàng ABBANK nhân s...   

                                         Description         Time_published  \
0  Với mong muốn cùng chung tay tiếp sức, hỗ trợ ...  25-09-2024 - 15:14 PM   
1  Vừa qua tại Hà Nội, Ngân hàng TMCP An Bình (A

1.1. Làm sạch văn bản

In [ ]:
# Làm sạch văn bản: loại bỏ ký tự đặc biệt, dấu câu và chuyển về chữ thường
def clean_text(text):
    # Kiểm tra xem text có phải là kiểu string hay không, nếu không thì chuyển đổi thành string
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'[^a-zA-Záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ\s]', '', text)  # Loại bỏ ký tự đặc biệt và số
    text = text.lower()  # Chuyển thành chữ thường
    return text

# Áp dụng hàm làm sạch cho cột 'Title' và 'Description'
df['Title'] = df['Title'].apply(clean_text)
df['Description'] = df['Description'].apply(clean_text)

print("Dữ liệu sau khi làm sạch:")
print(df[['Title', 'Description']].head())

Dữ liệu sau khi làm sạch:
                                               Title  \
0  abbank giảm tới năm lãi suất cho vay khách hàn...   
1  maybank tăng cường quan hệ hợp tác chiến lược ...   
2   cá nhân  tổ chức sở hữu gần  vốn điều lệ ngân...   
3  abbank đạt  tỷ đồng lợi nhuận trước thuế trong...   
4  nhiều ưu đãi dành cho khách hàng abbank nhân s...   

                                         Description  
0  với mong muốn cùng chung tay tiếp sức hỗ trợ n...  
1  vừa qua tại hà nội ngân hàng tmcp an bình abba...  
2  ngân hàng tmcp an bình abbank  abb vừa công bố...  
3  kết thúc quý ii ngân hàng tmcp an bình abbank ...  
4  mừng sinh nhật  năm abbank đã triển khai nhiều...  


1.2: Tiền xử lý văn bản

Tokenization: Chúng ta sẽ tách các câu thành các từ nhỏ hơn.
Lemmatization/Stemming: Giảm từ về dạng gốc để giảm thiểu sự đa dạng.
Loại bỏ Stop Words: Loại bỏ các từ không mang nhiều ý nghĩa (ví dụ: “và,” “của,”...).

In [ ]:
#Tạo danh sách stop words tiếng Việt từ một file công khai:

import requests

url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-vi/master/stopwords-vi.txt"
response = requests.get(url)
vietnamese_stopwords = set(response.text.splitlines())

In [ ]:
# Cập nhật mã tiền xử lý văn bản bằng cách sử dụng danh sách vietnamese_stopwords:

# Tải xuống tài nguyên cần thiết từ NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab') # Download the missing punkt_tab resource

# Khởi tạo công cụ lemmatizer
lemmatizer = WordNetLemmatizer()

# Hàm tiền xử lý
def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in vietnamese_stopwords]  # Loại bỏ stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return tokens

# Áp dụng tiền xử lý cho cột 'Title' và 'Description'
df['Title'] = df['Title'].apply(preprocess_text)
df['Description'] = df['Description'].apply(preprocess_text)

print("Dữ liệu sau khi tiền xử lý:")
print(df[['Title', 'Description']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Dữ liệu sau khi tiền xử lý:
                                               Title  \
0  [abbank, giảm, lãi, suất, vay, khách, hàng, ản...   
1  [maybank, tăng, cường, quan, hệ, hợp, tác, chi...   
2  [cá, nhân, tổ, chức, sở, hữu, gần, vốn, lệ, ng...   
3  [abbank, đạt, tỷ, đồng, lợi, nhuận, thuế, hoàn...   
4  [ưu, đãi, dành, khách, hàng, abbank, nhân, sin...   

                                         Description  
0  [mong, chung, tay, tiếp, sức, hỗ, trợ, dân, ph...  
1  [hà, nội, ngân, hàng, tmcp, an, bình, abbank, ...  
2  [ngân, hàng, tmcp, an, bình, abbank, abb, công...  
3  [kết, thúc, quý, ii, ngân, hàng, tmcp, an, bìn...  
4  [mừng, sinh, nhật, abbank, triển, khai, ưu, đã...  


1.3 Mã hóa đánh giá

In [ ]:
def encode_evaluation(evaluation):
    # Kiểm tra nếu giá trị là float và chuyển đổi thành string trước khi xử lý
    if isinstance(evaluation, float):
        evaluation = str(evaluation)

    if evaluation.lower() == 'positive':
        return 1
    elif evaluation.lower() == 'negative':
        return 0
    else:
        return 2  # Neutral

# Áp dụng mã hóa
df['Evaluate(Human)'] = df['Evaluate( Human)'].apply(encode_evaluation)

# Hiển thị dữ liệu đã mã hóa
print(df['Evaluate( Human)'].head())


0    Positive
1    Positive
2     Neutral
3    Positive
4    Positive
Name: Evaluate( Human), dtype: object


1.4. Chia dữ liệu

In [ ]:
# Lọc ra các dòng có nhãn (cột 'Evaluate( Human)' không bị NaN)
df_labeled = df[df['Evaluate(Human)'].notna()]

# Đảm bảo rằng df_labeled chỉ có 6,000 dòng dữ liệu có nhãn
df_labeled = df_labeled.sample(n=6000, random_state=42)

# Tách các cột đặc trưng (Title, Description) và nhãn (Evaluate( Human))
X = df_labeled[['Title', 'Description']]
y = df_labeled['Evaluate(Human)']

# Chia dữ liệu thành tập huấn luyện và kiểm tra (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Phần dữ liệu không có nhãn để dự đoán sau
df_unlabeled = df[df['Evaluate(Human)'].isna()]

# Kết quả
print("Kích thước dữ liệu huấn luyện:", X_train.shape)
print("Kích thước dữ liệu kiểm tra:", X_test.shape)
print("Số lượng dòng không có nhãn để phân loại sau:", df_unlabeled.shape[0])

Kích thước dữ liệu huấn luyện: (4800, 2)
Kích thước dữ liệu kiểm tra: (1200, 2)
Số lượng dòng không có nhãn để phân loại sau: 0


Bước 2: Trích Xuất Tính Năng

In [ ]:
# Khởi tạo TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Áp dụng TF-IDF cho cột văn bản 'Title' và 'Description'
tfidf_title = tfidf_vectorizer.fit_transform(df['Title'].apply(lambda x: ' '.join(x)))
tfidf_description = tfidf_vectorizer.fit_transform(df['Description'].apply(lambda x: ' '.join(x)))

print("Shape of TF-IDF matrix for Title:", tfidf_title.shape)
print("Shape of TF-IDF matrix for Description:", tfidf_description.shape)

Shape of TF-IDF matrix for Title: (8079, 3298)
Shape of TF-IDF matrix for Description: (8079, 4038)


Chuyển đổi từ ngữ thành vector số sử dụng BERT

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Bước 1: Tải mô hình BERT và khởi tạo các hàm tính toán BERT embedding
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Chuyển đổi từ ngữ thành vector
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    # Lấy embedding từ hidden state cuối cùng
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


# Tính toán BERT embedding cho cả Title và Description
def compute_combined_bert_embedding(row):
    title_embedding = get_bert_embedding(' '.join(row['Title']))
    description_embedding = get_bert_embedding(' '.join(row['Description']))
    # Cách 1: Tính trung bình hai vector embedding
    combined_embedding = (title_embedding + description_embedding) / 2
    # Cách 2: Ghép nối hai vector (bỏ dấu # trước dòng dưới để sử dụng cách này)
    # combined_embedding = np.hstack((title_embedding, description_embedding))
    return combined_embedding

# Áp dụng hàm tính BERT embedding cho từng hàng trong DataFrame
X_bert = np.vstack(df.apply(compute_combined_bert_embedding, axis=1))
print("Shape of combined BERT embeddings:", X_bert.shape)

# Bước 2: Tính toán TF-IDF và ghép nối với BERT embedding
# Khởi tạo TfidfVectorizer cho Title và Description
tfidf_vectorizer_title = TfidfVectorizer()
tfidf_vectorizer_description = TfidfVectorizer()

# Tính TF-IDF cho Title và Description
tfidf_title = tfidf_vectorizer_title.fit_transform(df['Title'].apply(lambda x: ' '.join(x)))
tfidf_description = tfidf_vectorizer_description.fit_transform(df['Description'].apply(lambda x: ' '.join(x)))

# Ghép nối các đặc trưng TF-IDF và BERT embedding
X_combined = hstack([tfidf_title, tfidf_description, X_bert])

print("Shape of the final combined feature matrix:", X_combined.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Shape of combined BERT embeddings: (8079, 768)
Shape of the final combined feature matrix: (8079, 8104)


**SVM**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd

# Hàm kết hợp văn bản từ 'Title' và 'Description'
def combine_text(row):
    title = " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])
    return title + " " + description

# Tạo văn bản kết hợp cho tập huấn luyện và kiểm tra
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Vector hóa dữ liệu văn bản kết hợp bằng TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_combined)

# Áp dụng SMOTE trên dữ liệu đã vector hóa
smote = SMOTE(random_state=42, sampling_strategy='auto')
X_resampled, y_resampled = smote.fit_resample(X_train_vectorized, y_train)

# Xây dựng pipeline với SVM (TF-IDF đã được áp dụng bên ngoài)
model_svm = Pipeline([  # Lưu mô hình trong biến svm_model
    ('svm', SVC(kernel='linear', random_state=42))  # Bước phân loại với SVM
])

# Huấn luyện mô hình với dữ liệu đã cân bằng
model_svm.fit(X_resampled, y_resampled)

# Vector hóa dữ liệu kiểm tra
X_test_vectorized = vectorizer.transform(X_test_combined)

# Dự đoán trên tập kiểm tra
y_pred = model_svm.predict(X_test_vectorized)

# Đánh giá mô hình
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("\nBáo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))


Độ chính xác trên tập kiểm tra: 0.715

Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.80      0.77      0.79       393
           1       0.69      0.72      0.71       418
           2       0.65      0.66      0.65       389

    accuracy                           0.71      1200
   macro avg       0.72      0.71      0.72      1200
weighted avg       0.72      0.71      0.72      1200



In [ ]:
# Huấn luyện mô hình
model_svm.fit(X_resampled, y_resampled)

Pipeline(steps=[('svm', SVC(kernel='linear', random_state=42))])

In [ ]:
def predict_sentiment(title, description):
    """
    Hàm này nhận đầu vào là `title` và `description`,
    kết hợp chúng lại, vector hóa và dự đoán nhãn phân loại.
    """
    # Kết hợp title và description
    combined_text = f"{title} {description}"

    # Dự đoán nhãn bằng mô hình đã huấn luyện
    prediction = model_svm.predict([combined_text])[0]

    return prediction

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

# Tích hợp đầy đủ TfidfVectorizer và SVM vào pipeline
model_svm = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),  # Vector hóa TF-IDF
    ('svm', SVC(kernel='linear', probability=True, random_state=42))  # Mô hình SVM
])

In [ ]:
# Huấn luyện mô hình pipeline
model_svm.fit(X_train_combined, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=10000)),
                ('svm',
                 SVC(kernel='linear', probability=True, random_state=42))])

In [ ]:
def predict_sentiment(title, description):
    # Kết hợp title và description
    combined_text = f"{title} {description}"

    # Dự đoán trực tiếp với pipeline
    prediction = model_svm.predict([combined_text])[0]

    return prediction

In [ ]:
# Dữ liệu thử nghiệm
title = "Công nghệ AI đang thay đổi thế giới"
description = "Trí tuệ nhân tạo có nhiều lợi ích nhưng cũng có rủi ro"

# Gọi hàm dự đoán
result = predict_sentiment(title, description)

# In kết quả
print(f"Kết quả phân loại: {result}")
# Từ điển ánh xạ nhãn
label_mapping = {
    0: "Negative",
    1: "Positive",
    2: "Neutral"
}

# Hiển thị nhãn dạng văn bản
result_text = label_mapping[result]
print(f"Kết quả phân loại: {result_text}")

Kết quả phân loại: 2
Kết quả phân loại: Neutral


In [ ]:
# Dữ liệu thử nghiệm (Bài báo tài chính)
title = "Ngân hàng Trung Ương Tăng Lãi Suất Lần Thứ Ba Trong Năm"
description = ("Ngân hàng Trung Ương vừa công bố tăng lãi suất cơ bản lên 6.5% nhằm kiểm soát lạm phát, "
               "đánh dấu lần tăng thứ ba trong năm nay. Quyết định này được đưa ra sau khi lạm phát chạm mức 8% "
               "trong tháng trước, mức cao nhất trong vòng 20 năm. Thị trường chứng khoán đã phản ứng tiêu cực, "
               "với chỉ số VN-Index giảm 3% ngay sau thông báo. Các chuyên gia dự đoán việc tăng lãi suất sẽ tiếp "
               "tục ảnh hưởng đến thị trường bất động sản và chi phí vay vốn của doanh nghiệp trong thời gian tới.")

# Gọi hàm dự đoán
result = predict_sentiment(title, description)

# In kết quả dự đoán dạng số
print(f"Kết quả phân loại (dạng số): {result}")

# Từ điển ánh xạ nhãn
label_mapping = {
    0: "Negative",  # Tiêu cực
    1: "Positive",  # Tích cực
    2: "Neutral"    # Trung lập
}

# Hiển thị nhãn dạng văn bản
result_text = label_mapping[result]
print(f"Kết quả phân loại (dạng văn bản): {result_text}")


Kết quả phân loại (dạng số): 1
Kết quả phân loại (dạng văn bản): Positive


In [ ]:
# Các dữ liệu thử nghiệm về bài báo tài chính
examples = [
    {
        "title": "Thị Trường Chứng Khoán Tăng Điểm Mạnh Sau Báo Cáo Tăng Trưởng GDP",
        "description": "Chỉ số VN-Index đã tăng 2.5% trong phiên giao dịch hôm nay sau khi báo cáo GDP quý IV cho thấy mức tăng trưởng 6.8%, vượt xa kỳ vọng của các chuyên gia. Nhóm cổ phiếu ngân hàng và bất động sản dẫn đầu đà tăng. Các nhà đầu tư lạc quan rằng nền kinh tế đang phục hồi mạnh mẽ sau đại dịch."
    },
    {
        "title": "Phát hiện từ 5 tờ tiền giả trở lên trong một giao dịch, ngân hàng phải báo ngay cho công an",
        "description": "Đây là quy định tại Thông tư 58/2024/TT-NHNN do Ngân hàng Nhà nước ban hành mới đây."
    },
    {
        "title": "Doanh Thu Xuất Khẩu Nông Sản Đạt Kỷ Lục Trong Năm 2023",
        "description": "Theo báo cáo từ Bộ Nông nghiệp, giá trị xuất khẩu nông sản Việt Nam đạt 55 tỷ USD trong năm 2023, cao nhất từ trước đến nay. Các mặt hàng chủ lực như gạo, cà phê và hạt điều đều ghi nhận mức tăng trưởng mạnh, đặc biệt tại các thị trường châu Âu và Mỹ."
    },
    {
        "title": "Doanh Nghiệp Công Nghệ Đối Mặt Khó Khăn Tài Chính",
        "description": "Nhiều công ty công nghệ Việt Nam đang phải cắt giảm nhân sự và chi phí sau khi không huy động được vốn mới từ các quỹ đầu tư. Nguyên nhân chính đến từ sự sụt giảm niềm tin của nhà đầu tư và áp lực từ việc tăng lãi suất toàn cầu. Một số chuyên gia cảnh báo rằng ngành công nghệ sẽ tiếp tục đối mặt với thách thức trong 12 tháng tới."
    },
    {
        "title": "Cận Tết Nguyên đán, một ngân hàng thông báo chấm dứt hoạt động 3 phòng giao dịch",
        "description": "Ngân hàng TMCP Sài Gòn (SCB) vừa thông báo chấm dứt hoạt động 3 phòng giao dịch tại TP HCM, Khánh Hòa và Bà Rịa Vũng Tàu."
    }
]

# Từ điển ánh xạ nhãn
label_mapping = {
    0: "Negative",  # Tiêu cực
    1: "Positive",  # Tích cực
    2: "Neutral"    # Trung lập
}

# Dự đoán và hiển thị kết quả cho từng ví dụ
for idx, example in enumerate(examples):
    title = example["title"]
    description = example["description"]

    # Gọi hàm dự đoán
    result = predict_sentiment(title, description)

    # Hiển thị nhãn dạng văn bản
    result_text = label_mapping[result]

    # In kết quả
    print(f"Ví dụ {idx + 1}:")
    print(f"Tiêu đề: {title}")
    print(f"Mô tả: {description}")
    print(f"Kết quả phân loại: {result_text}\n")


Ví dụ 1:
Tiêu đề: Thị Trường Chứng Khoán Tăng Điểm Mạnh Sau Báo Cáo Tăng Trưởng GDP
Mô tả: Chỉ số VN-Index đã tăng 2.5% trong phiên giao dịch hôm nay sau khi báo cáo GDP quý IV cho thấy mức tăng trưởng 6.8%, vượt xa kỳ vọng của các chuyên gia. Nhóm cổ phiếu ngân hàng và bất động sản dẫn đầu đà tăng. Các nhà đầu tư lạc quan rằng nền kinh tế đang phục hồi mạnh mẽ sau đại dịch.
Kết quả phân loại: Positive

Ví dụ 2:
Tiêu đề: Phát hiện từ 5 tờ tiền giả trở lên trong một giao dịch, ngân hàng phải báo ngay cho công an
Mô tả: Đây là quy định tại Thông tư 58/2024/TT-NHNN do Ngân hàng Nhà nước ban hành mới đây.
Kết quả phân loại: Neutral

Ví dụ 3:
Tiêu đề: Doanh Thu Xuất Khẩu Nông Sản Đạt Kỷ Lục Trong Năm 2023
Mô tả: Theo báo cáo từ Bộ Nông nghiệp, giá trị xuất khẩu nông sản Việt Nam đạt 55 tỷ USD trong năm 2023, cao nhất từ trước đến nay. Các mặt hàng chủ lực như gạo, cà phê và hạt điều đều ghi nhận mức tăng trưởng mạnh, đặc biệt tại các thị trường châu Âu và Mỹ.
Kết quả phân loại: Positive

Ví

In [ ]:
import joblib

# Lưu mô hình
joblib.dump(model_svm, 'svm_sentiment_model.pkl')

# Tải lại mô hình
model_svm = joblib.load('svm_sentiment_model.pkl')


**CNN**

In [ ]:
pip install tensorflow keras

CNN.1 Chuyển đổi văn bản thành chuỗi số

Để huấn luyện CNN, ta sẽ sử dụng Tokenizer từ Keras để chuyển văn bản thành chuỗi số và pad_sequences để làm cho tất cả các chuỗi có độ dài cố định.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Kết hợp 'Title' và 'Description' thành văn bản duy nhất
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Khởi tạo Tokenizer và fit vào dữ liệu huấn luyện
tokenizer = Tokenizer(num_words=10000)  # Sử dụng 10,000 từ phổ biến nhất
tokenizer.fit_on_texts(X_train_combined)

# Chuyển văn bản thành chuỗi số và làm cho chúng có độ dài cố định
X_train_seq = tokenizer.texts_to_sequences(X_train_combined)
X_test_seq = tokenizer.texts_to_sequences(X_test_combined)

# Đảm bảo tất cả chuỗi có cùng độ dài (ví dụ: maxlen=100)
maxlen = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

CNN.2 Xây dựng mô hình CNN

Mô hình CNN sẽ bao gồm lớp nhúng (Embedding layer) để chuyển các từ thành vector có kích thước cố định, và sau đó là các lớp tích chập (Conv1D), lớp gộp (MaxPooling1D), và lớp kết nối hoàn toàn (Dense) để phân loại.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Khởi tạo tham số
embedding_dim = 100  # Kích thước của vector nhúng
num_classes = len(y_train.unique())  # Số lớp đầu ra (tùy vào bài toán)

# Xây dựng mô hình CNN
model_cnn = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=maxlen),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Sử dụng softmax cho phân loại nhiều lớp
])

# Biên dịch mô hình
model_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


CNN.3 Huấn luyện mô hình

Huấn luyện mô hình với dữ liệu đã chuẩn bị:

In [ ]:
# Huấn luyện mô hình
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)  # Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra


# ... (rest of your code) ...

# Huấn luyện mô hình với y_train_encoded
history = model_cnn.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - accuracy: 0.4122 - loss: 1.0653 - val_accuracy: 0.5802 - val_loss: 0.9173
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.6220 - loss: 0.8255 - val_accuracy: 0.7104 - val_loss: 0.6964
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - accuracy: 0.7508 - loss: 0.5826 - val_accuracy: 0.7094 - val_loss: 0.6723
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - accuracy: 0.8494 - loss: 0.3998 - val_accuracy: 0.7052 - val_loss: 0.7215
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - accuracy: 0.8994 - loss: 0.2975 - val_accuracy: 0.7219 - val_loss: 0.7885
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 81ms/step - accuracy: 0.9242 - loss: 0.2265 - val_accuracy: 0.7229 - val_loss: 0.9428
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - accuracy: 0.9620 - loss: 0.1299 - val_accuracy: 0.6990 - val_loss: 1.1103
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - accuracy: 0.9703 - loss: 0.0975 - val_accuracy: 0.7146 -

CNN.4 Đánh giá mô hình

Sau khi mô hình đã được huấn luyện, chúng ta sẽ đánh giá trên tập kiểm tra.

In [ ]:
# Đánh giá trên tập kiểm tra
# Import LabelEncoder if not already imported
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder nếu chưa được khởi tạo
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi nếu chưa được thực hiện
y_train_encoded = label_encoder.fit_transform(y_train)

# Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra
y_test_encoded = label_encoder.transform(y_test)

test_loss, test_accuracy = model_cnn.evaluate(X_test_pad, y_test_encoded) # Change y_test to y_test_encoded
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7103 - loss: 1.5111
Độ chính xác trên tập kiểm tra: 0.6891666650772095


CNN.5 Dự đoán và báo cáo phân loại

Cuối cùng, sử dụng mô hình đã huấn luyện để tạo ra dự đoán và báo cáo phân loại chi tiết.


In [ ]:
from sklearn.metrics import classification_report

# Dự đoán trên tập kiểm tra
y_pred = model_cnn.predict(X_test_pad)
y_pred_classes = y_pred.argmax(axis=1)  # Chuyển xác suất thành nhãn

# Chuyển đổi y_pred_classes về dạng nhãn gốc trước khi tính toán báo cáo
y_pred_classes = label_encoder.inverse_transform(y_pred_classes)

# Báo cáo phân loại
print("Báo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred_classes))

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.82      0.73      0.77       393
           1       0.66      0.74      0.70       418
           2       0.61      0.60      0.60       389

    accuracy                           0.69      1200
   macro avg       0.70      0.69      0.69      1200
weighted avg       0.69      0.69      0.69      1200



**LSTM**

In [ ]:
pip install tensorflow keras

LSTM.1 Chuẩn bị dữ liệu cho LSTM

Sử dụng lại phần chuẩn bị dữ liệu đã thực hiện cho mô hình CNN. Ta sẽ sử dụng Tokenizer và pad_sequences từ Keras để chuyển văn bản thành chuỗi số và định dạng chiều dài.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Kết hợp 'Title' và 'Description' thành văn bản duy nhất
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Khởi tạo Tokenizer và fit vào dữ liệu huấn luyện
tokenizer = Tokenizer(num_words=10000)  # Sử dụng 10,000 từ phổ biến nhất
tokenizer.fit_on_texts(X_train_combined)

# Chuyển văn bản thành chuỗi số và làm cho chúng có độ dài cố định
X_train_seq = tokenizer.texts_to_sequences(X_train_combined)
X_test_seq = tokenizer.texts_to_sequences(X_test_combined)

# Đảm bảo tất cả chuỗi có cùng độ dài (ví dụ: maxlen=100)
maxlen = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

LSTM.2 Xây dựng mô hình LSTM

Mô hình LSTM sẽ bao gồm lớp nhúng (Embedding layer), lớp LSTM, lớp gộp (Dropout), và lớp kết nối hoàn toàn (Dense) để phân loại.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Khởi tạo tham số
embedding_dim = 100  # Kích thước của vector nhúng
num_classes = len(y_train.unique())  # Số lớp đầu ra (tùy vào bài toán)

# Xây dựng mô hình LSTM
model_lstm = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=maxlen),
    LSTM(128, return_sequences=False),  # LSTM với 128 đơn vị
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Sử dụng softmax cho phân loại nhiều lớp
])

# Biên dịch mô hình
model_lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


LSTM.3 Huấn luyện mô hình
Huấn luyện mô hình với dữ liệu đã chuẩn bị:

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Fit và transform y_train
y_train_encoded = label_encoder.fit_transform(y_train)

# Huấn luyện mô hình
history = model_lstm.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 260ms/step - accuracy: 0.3316 - loss: 1.1003 - val_accuracy: 0.3229 - val_loss: 1.0994
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 15s 257ms/step - accuracy: 0.3375 - loss: 1.0990 - val_accuracy: 0.3365 - val_loss: 1.0990
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 19s 226ms/step - accuracy: 0.3357 - loss: 1.0992 - val_accuracy: 0.3229 - val_loss: 1.1020
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 16s 145ms/step - accuracy: 0.3489 - loss: 1.0995 - val_accuracy: 0.3406 - val_loss: 1.0985
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 155ms/step - accuracy: 0.3184 - loss: 1.0996 - val_accuracy: 0.3229 - val_loss: 1.0992
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 154ms/step - accuracy: 0.3426 - loss: 1.0996 - val_accuracy: 0.3229 - val_loss: 1.0987
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 8s 138ms/step - accuracy: 0.3237 - loss: 1.1000 - val_accuracy: 0.3229 - val_loss: 1.0992
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 9s 151ms/step - accuracy: 0.3316 - loss: 1.0994 - val_accurac

LSTM.4 Đánh giá mô hình

Sau khi mô hình đã được huấn luyện, chúng ta sẽ đánh giá trên tập kiểm tra



In [ ]:
# Đánh giá trên tập kiểm tra
# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Fit và transform y_train (đã thực hiện trước đó)
y_train_encoded = label_encoder.fit_transform(y_train)

# Huấn luyện mô hình (đã thực hiện trước đó)
# history = model.fit(X_train_pad, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)

# ----> Transform y_test using the same LabelEncoder <----
y_test_encoded = label_encoder.transform(y_test)

# Đánh giá trên tập kiểm tra
test_loss, test_accuracy = model_lstm.evaluate(X_test_pad, y_test_encoded)  # Use y_test_encode
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.3299 - loss: 1.0988
Độ chính xác trên tập kiểm tra: 0.3241666555404663


LSTM.5 Dự đoán và báo cáo phân loại

In [ ]:
from sklearn.metrics import classification_report

# Dự đoán trên tập kiểm tra
y_pred = model_lstm.predict(X_test_pad)
y_pred_classes = y_pred.argmax(axis=1)  # Chuyển xác suất thành nhãn

# ----> Transform y_pred_classes back to original labels <----
y_pred_classes = label_encoder.inverse_transform(y_pred_classes)

# Báo cáo phân loại
print("Báo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred_classes))

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step
Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       393
           1       0.00      0.00      0.00       418
           2       0.32      1.00      0.49       389

    accuracy                           0.32      1200
   macro avg       0.11      0.33      0.16      1200
weighted avg       0.11      0.32      0.16      1200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**PhoBERT**

In [ ]:
pip install transformers torch scikit-learn


PhoBert.1 Chuẩn bị dữ liệu

Giống như trước, chúng ta sẽ kết hợp cột "Title" và "Description", và sau đó chuẩn bị dữ liệu cho PhoBERT.

In [ ]:
import pandas as pd

# Kết hợp 'Title' và 'Description' thành văn bản duy nhất
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Đặt nhãn cho dữ liệu
y_train = y_train.values
y_test = y_test.values

PhoBert.2 Tải mô hình PhoBERT và Tokenizer

Tải mô hình PhoBERT từ Hugging Face.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Tải Tokenizer và Mô hình PhoBERT
tokenizer = BertTokenizer.from_pretrained("vinai/phobert-base")
model_phobert = BertForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=len(set(y_train)))

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.w

PhoBert.3 Mã hóa văn bản

Sử dụng tokenizer để mã hóa văn bản đầu vào thành định dạng mà mô hình có thể hiểu được.

In [ ]:
# Mã hóa dữ liệu
train_encodings = tokenizer(list(X_train_combined), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test_combined), truncation=True, padding=True, max_length=128)

PhoBert.4 Tạo Dataset

Tạo dataset cho huấn luyện mô hình.

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tạo dataset cho tập huấn luyện và kiểm tra
train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)


PhoBert.5 Huấn luyện mô hình

Thiết lập các tham số huấn luyện và tiến hành huấn luyện mô hình.

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels  # Using encoded labels directly

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Ensure labels is an integer by casting to int before creating the tensor
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)

# Khởi tạo dataset với nhãn đã mã hóa (Using encoded labels)
train_dataset = CustomDataset(train_encodings, y_train_encoded)
test_dataset = CustomDataset(test_encodings, y_test_encoded)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01
)

# Trainer initialization
trainer = Trainer(
    model=model_phobert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,1.090300,1.074762
2,1.050600,1.016791
3,1.031800,1.014762


TrainOutput(global_step=1800, training_loss=1.0511624993218316, metrics={'train_runtime': 10779.3306, 'train_samples_per_second': 1.336, 'train_steps_per_second': 0.167, 'total_flos': 673405903497600.0, 'train_loss': 1.0511624993218316, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred  # Lấy logits và nhãn
    predictions = logits.argmax(axis=-1)  # Dự đoán nhãn từ logits
    accuracy = accuracy_score(labels, predictions)  # Tính độ chính xác
    return {'eval_accuracy': accuracy}  # Trả về từ điển với độ chính xác

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder nếu chưa được khởi tạo
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi nếu chưa được thực hiện
y_train_encoded = label_encoder.fit_transform(y_train)

# Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra
y_test_encoded = label_encoder.transform(y_test)

# Tạo đối tượng Trainer mới với hàm compute_metrics
trainer = Trainer(
    model=model_phobert,  # Mô hình đã huấn luyện
    args=training_args,  # Tham số huấn luyện
    train_dataset=train_dataset,  # Tập huấn luyện
    eval_dataset=test_dataset,  # Tập kiểm tra
    compute_metrics=compute_metrics  # Thêm hàm compute_metrics
)

# Đánh giá mô hình
eval_results = trainer.evaluate()  # Đánh giá mô hình

# In kết quả
print("Loss trên tập kiểm tra:", eval_results['eval_loss'])
print("Độ chính xác trên tập kiểm tra:", eval_results['eval_accuracy'])


Loss trên tập kiểm tra: 1.0147618055343628
Độ chính xác trên tập kiểm tra: 0.49333333333333335


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Khởi tạo LabelEncoder nếu chưa được khởi tạo
label_encoder = LabelEncoder()

# Fit LabelEncoder vào dữ liệu huấn luyện và biến đổi nếu chưa được thực hiện
y_train_encoded = label_encoder.fit_transform(y_train)

# Sử dụng cùng LabelEncoder cho dữ liệu kiểm tra
y_test_encoded = label_encoder.transform(y_test)

# Tạo đối tượng Trainer mới với hàm compute_metrics
trainer = Trainer(
    model=model_phobert,  # Mô hình đã huấn luyện
    args=training_args,  # Tham số huấn luyện
    train_dataset=train_dataset,  # Tập huấn luyện
    eval_dataset=test_dataset,  # Tập kiểm tra
    compute_metrics=compute_metrics  # Thêm hàm compute_metrics
)

# Đánh giá mô hình
eval_results = trainer.evaluate()  # Đánh giá mô hình

# In kết quả Loss và Độ chính xác
print("Loss trên tập kiểm tra:", eval_results['eval_loss'])
print("Độ chính xác trên tập kiểm tra:", eval_results['eval_accuracy'])

# Dự đoán trên tập kiểm tra
y_pred = trainer.predict(test_dataset).predictions
y_pred_labels = np.argmax(y_pred, axis=1)

# Convert label_encoder.classes_ to a list of strings
target_names = [str(cls) for cls in label_encoder.classes_]  # Convert to strings

# In báo cáo phân loại chi tiết
print("Báo cáo phân loại:\n", classification_report(y_test_encoded, y_pred_labels, target_names=target_names)) # Use target_names

Loss trên tập kiểm tra: 1.0147618055343628
Độ chính xác trên tập kiểm tra: 0.49333333333333335
Báo cáo phân loại:
               precision    recall  f1-score   support

           0       0.65      0.35      0.45       393
           1       0.48      0.64      0.55       418
           2       0.43      0.49      0.46       389

    accuracy                           0.49      1200
   macro avg       0.52      0.49      0.49      1200
weighted avg       0.52      0.49      0.49      1200



**Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd

# Hàm kết hợp văn bản từ 'Title' và 'Description'
def combine_text(row):
    title = " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])
    return title + " " + description

# Tạo văn bản kết hợp cho tập huấn luyện và kiểm tra
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Vector hóa dữ liệu văn bản kết hợp bằng TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train_combined)

# Áp dụng SMOTE trên dữ liệu đã vector hóa
smote = SMOTE(random_state=42, sampling_strategy='auto')
X_resampled, y_resampled = smote.fit_resample(X_train_vectorized, y_train)

# Xây dựng pipeline với Random Forest
model_rf = Pipeline([
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))  # Mô hình Random Forest
])

# Huấn luyện mô hình với dữ liệu đã cân bằng
model_rf.fit(X_resampled, y_resampled)

# Vector hóa dữ liệu kiểm tra
X_test_vectorized = vectorizer.transform(X_test_combined)

# Dự đoán trên tập kiểm tra
y_pred = model_rf.predict(X_test_vectorized)

# Đánh giá mô hình
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("\nBáo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))

Độ chính xác trên tập kiểm tra: 0.7158333333333333

Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.82      0.74      0.78       393
           1       0.68      0.75      0.71       418
           2       0.65      0.66      0.66       389

    accuracy                           0.72      1200
   macro avg       0.72      0.71      0.72      1200
weighted avg       0.72      0.72      0.72      1200



**GMM**

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Hàm kết hợp văn bản từ 'Title' và 'Description'
def combine_text(row):
    title = " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])
    return title + " " + description

# Tạo văn bản kết hợp cho tập huấn luyện và kiểm tra
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Vector hóa dữ liệu văn bản kết hợp bằng TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_vectorized = vectorizer.fit_transform(X_train_combined)
X_test_vectorized = vectorizer.transform(X_test_combined)

# Huấn luyện mô hình GMM
num_classes = len(np.unique(y_train))  # Số lượng nhãn
gmm_model = GaussianMixture(n_components=num_classes, random_state=42)
gmm_model.fit(X_train_vectorized.toarray())

# Dự đoán trên tập kiểm tra
y_pred = gmm_model.predict(X_test_vectorized.toarray())

# Đánh giá mô hình
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("\nBáo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))


Độ chính xác trên tập kiểm tra: 0.275

Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.24      0.53      0.33       393
           1       0.61      0.10      0.17       418
           2       0.30      0.20      0.24       389

    accuracy                           0.28      1200
   macro avg       0.38      0.28      0.25      1200
weighted avg       0.39      0.28      0.25      1200



**Naive Bayes**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Hàm kết hợp văn bản, xử lý cả list trong cột 'Title' và 'Description'
def combine_text(row):
    """Combines 'Title' and 'Description' handling potential lists in both columns."""
    # Chuyển 'Title' thành chuỗi nếu có dạng list hoặc list lồng
    title = " ".join([str(item) for sublist in row['Title'] for item in sublist]) if isinstance(row['Title'], list) and any(isinstance(i, list) for i in row['Title']) else " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])

    # Chuyển 'Description' thành chuỗi nếu có dạng list
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])

    return title + " " + description

# Áp dụng hàm để tạo đặc trưng văn bản kết hợp
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Xây dựng pipeline với TF-IDF và Naive Bayes
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Bước chuyển văn bản thành TF-IDF
    ('nb', MultinomialNB())  # Bước phân loại với Naive Bayes
])

# Huấn luyện mô hình
pipeline.fit(X_train_combined, y_train)

# Dự đoán trên tập kiểm tra
y_pred = pipeline.predict(X_test_combined)

# Đánh giá mô hình
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("\nBáo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))

Độ chính xác trên tập kiểm tra: 0.6816666666666666

Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.84      0.72      0.77       393
           1       0.64      0.70      0.67       418
           2       0.60      0.63      0.61       389

    accuracy                           0.68      1200
   macro avg       0.69      0.68      0.68      1200
weighted avg       0.69      0.68      0.68      1200



**Logistic Regression**


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Hàm kết hợp văn bản, xử lý cả list trong cột 'Title' và 'Description'
def combine_text(row):
    """Combines 'Title' and 'Description' handling potential lists in both columns."""
    # Chuyển 'Title' thành chuỗi nếu có dạng list hoặc list lồng
    title = " ".join([str(item) for sublist in row['Title'] for item in sublist]) if isinstance(row['Title'], list) and any(isinstance(i, list) for i in row['Title']) else " ".join(row['Title']) if isinstance(row['Title'], list) else str(row['Title'])

    # Chuyển 'Description' thành chuỗi nếu có dạng list
    description = " ".join(row['Description']) if isinstance(row['Description'], list) else str(row['Description'])

    return title + " " + description

# Áp dụng hàm để tạo đặc trưng văn bản kết hợp
X_train_combined = X_train.apply(combine_text, axis=1)
X_test_combined = X_test.apply(combine_text, axis=1)

# Xây dựng pipeline với TF-IDF và Logistic Regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Bước chuyển văn bản thành TF-IDF
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))  # Sử dụng Logistic Regression
])

# Huấn luyện mô hình
pipeline.fit(X_train_combined, y_train)

# Dự đoán trên tập kiểm tra
y_pred = pipeline.predict(X_test_combined)

# Đánh giá mô hình
print("Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
print("Báo cáo phân loại chi tiết:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Độ chính xác trên tập kiểm tra: 0.705
Báo cáo phân loại chi tiết:
               precision    recall  f1-score   support

           0       0.80      0.77      0.79       393
           1       0.69      0.68      0.68       418
           2       0.63      0.67      0.65       389

    accuracy                           0.70      1200
   macro avg       0.71      0.71      0.71      1200
weighted avg       0.71      0.70      0.71      1200

Confusion Matrix:
 [[304  39  50]
 [ 34 283 101]
 [ 41  89 259]]
